# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path

In [3]:
print(tf.__version__)

2.5.0


In [4]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
if tf.test.gpu_device_name(): 
  
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
#makes all outputs be in float format rather than exponentials
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [7]:
#Set Parameters
IMG_SIZE = 224 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
log_dir=f'logs\\{MODEL_NAME}'
TENSORBOARD = TensorBoard(log_dir)

In [8]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [9]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
gan_dir = 'E:/Github/thesis/flowerDataset/new_GAN_dataset/'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
loaded_images_dir = 'E:/Dissertation/data/GANloaded_images.npy'
image_train_dir ='E:/Dissertation/data/GANimage_train.npy'
label_train_dir = 'E:/Dissertation/data/GANlabel_train.npy'
image_test_dir = 'E:/Dissertation/data/GANimage_test.npy'
label_test_dir = 'E:/Dissertation/data/GANlabel_test.npy'
image_val_dir = 'E:/Dissertation/data/GANimage_val.npy'
label_val_dir = 'E:/Dissertation/data/GANlabel_val.npy'

In [10]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)
df = pd.DataFrame(data_labels)
labels = df[1][3][0] #loads the labels that is stored in dataframe format
UNIQUE_LABELS = np.unique(labels)
UNIQUE_LABELS

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102], dtype=uint8)

In [11]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [12]:
load_images_from_folder(URL_dir)

[array([[[ 95, 119, 125],
         [ 87, 111, 117],
         [ 71,  99, 104],
         ...,
         [  6,  19,  17],
         [ 11,  24,  22],
         [ 16,  29,  27]],
 
        [[103, 125, 131],
         [ 91, 116, 122],
         [ 72, 102, 107],
         ...,
         [  2,  15,  13],
         [  6,  19,  17],
         [ 10,  23,  21]],
 
        [[112, 134, 139],
         [ 95, 123, 127],
         [ 72, 108, 110],
         ...,
         [  0,  12,  10],
         [  3,  16,  14],
         [  6,  19,  17]],
 
        ...,
 
        [[115, 147, 136],
         [101, 139, 129],
         [ 86, 129, 120],
         ...,
         [ 33,  50,  69],
         [ 35,  52,  72],
         [ 36,  53,  72]],
 
        [[113, 153, 145],
         [ 98, 142, 132],
         [ 87, 135, 123],
         ...,
         [ 30,  47,  66],
         [ 35,  52,  71],
         [ 34,  51,  70]],
 
        [[114, 155, 148],
         [ 99, 145, 134],
         [ 88, 138, 126],
         ...,
         [ 30,  47,  66],
  

In [13]:
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [14]:
#combine image with labels
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [15]:
image_data, labels = randomised_dataset(image_data, labels)

[   0    1    2 ... 8186 8187 8188]


In [16]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [17]:
split_data(image_data, labels)

image_train (6141, 224, 224, 3)
label_train (6141,)
image_test (820, 224, 224, 3)
label_test (820,)
image_val (1228, 224, 224, 3)
label_val (1228,)


In [18]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [19]:
#add GAN images

In [20]:
def add_gan(gan_dir):
    images = []
    labels = []
    for folder in os.listdir(gan_dir):
        full_path = os.path.join(gan_dir, folder)
        for file in os.listdir(full_path):
            img = cv2.resize(cv2.imread(os.path.join(full_path,file), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
            img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
            if img is not None:
                images.append(img)
                index = int(folder)
                labels.append(index)
                #print(labels)
    return images, labels


        

In [21]:
gan_images, gan_labels = add_gan(gan_dir)

In [22]:
gan_images = np.array(gan_images)

In [23]:
gan_labels = np.array(gan_labels)

In [24]:
gan_labels

array([ 1,  1,  1, ..., 99, 99, 99])

In [25]:
image_train = np.concatenate((image_train, gan_images), axis=0)

In [26]:
label_train = np.concatenate((label_train, gan_labels),axis=0)

In [27]:
print(image_train.shape)
print(label_train.shape)

(8283, 224, 224, 3)
(8283,)


In [28]:
mlb = LabelBinarizer()
label_train = np.array(mlb.fit_transform(label_train))
label_test = np.array(mlb.fit_transform(label_test))
label_val = np.array(mlb.fit_transform(label_val))

In [29]:
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [30]:
print(image_train.shape)
print(image_test.shape)
print(image_val.shape)

(8283, 224, 224, 3)
(820, 224, 224, 3)
(1228, 224, 224, 3)


# Custom CNN model

In [31]:
batch_size = 64
epochs = 100
verbose= 2
act = 'relu' #sigmoid tanh
dropout_val = 0.4
NUM_CLASSES = 102

In [32]:
from tensorflow.keras.applications  import ResNet152V2
from tensorflow.keras import datasets, layers, models, losses, Model

In [33]:
resnet = ResNet152V2(
    input_shape=(224,224,3),
    weights="imagenet",
    include_top=False, # removes last layer of the resnet model
)

In [34]:
for layer in resnet.layers: #don't train existing weights
    layer.trainable = False

In [35]:
x = layers.Flatten()(resnet.output)
#x = layers.Dense(1000, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [37]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Evaluation

In [38]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [40]:
EXPERIMENT_NAME = "GAN"
RUN_NAME = "ResNet152GAN_3"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        
        
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.keras.log_model(model, RUN_NAME)        

2021/08/22 11:51:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
C:\Users\yisi9\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/100
130/130 - 35s - loss: 5.8360 - accuracy: 0.6679 - val_loss: 4.3823 - val_accuracy: 0.7223
Epoch 2/100
130/130 - 21s - loss: 0.5226 - accuracy: 0.9534 - val_loss: 4.6372 - val_accuracy: 0.7272
Epoch 3/100
130/130 - 21s - loss: 0.2371 - accuracy: 0.9755 - val_loss: 4.5548 - val_accuracy: 0.7378
Epoch 4/100
130/130 - 21s - loss: 0.1350 - accuracy: 0.9877 - val_loss: 4.7911 - val_accuracy: 0.7508
Epoch 5/100
130/130 - 21s - loss: 0.1853 - accuracy: 0.9816 - val_loss: 5.4564 - val_accuracy: 0.7362
Epoch 6/100
130/130 - 21s - loss: 0.3709 - accuracy: 0.9737 - val_loss: 6.4796 - val_accuracy: 0.7378
Epoch 7/100
130/130 - 21s - loss: 0.3054 - accuracy: 0.9791 - val_loss: 7.6375 - val_accuracy: 0.7223
Epoch 8/100
130/130 - 21s - loss: 0.3016 - accuracy: 0.9790 - val_loss: 6.7357 - val_accuracy: 0.7459
Epoch 9/100
130/130 - 21s - loss: 0.2231 - accuracy: 0.9826 - val_loss: 7.7199 - val_accuracy: 0.7427
Epoch 10/100
130/130 - 21s - loss: 0.1664 - accuracy: 0.9866 - val_loss: 6.2137 - 

Epoch 79/100
130/130 - 21s - loss: 1.3672e-09 - accuracy: 1.0000 - val_loss: 10.8384 - val_accuracy: 0.7940
Epoch 80/100
130/130 - 21s - loss: 1.2953e-09 - accuracy: 1.0000 - val_loss: 10.8382 - val_accuracy: 0.7940
Epoch 81/100
130/130 - 21s - loss: 1.2665e-09 - accuracy: 1.0000 - val_loss: 10.8381 - val_accuracy: 0.7940
Epoch 82/100
130/130 - 21s - loss: 1.2089e-09 - accuracy: 1.0000 - val_loss: 10.8380 - val_accuracy: 0.7940
Epoch 83/100
130/130 - 21s - loss: 1.1658e-09 - accuracy: 1.0000 - val_loss: 10.8378 - val_accuracy: 0.7940
Epoch 84/100
130/130 - 21s - loss: 1.1226e-09 - accuracy: 1.0000 - val_loss: 10.8377 - val_accuracy: 0.7940
Epoch 85/100
130/130 - 21s - loss: 1.0218e-09 - accuracy: 1.0000 - val_loss: 10.8375 - val_accuracy: 0.7940
Epoch 86/100
130/130 - 21s - loss: 1.0074e-09 - accuracy: 1.0000 - val_loss: 10.8374 - val_accuracy: 0.7940
Epoch 87/100
130/130 - 21s - loss: 9.6427e-10 - accuracy: 1.0000 - val_loss: 10.8372 - val_accuracy: 0.7940
Epoch 88/100
130/130 - 21s -

C:\Users\yisi9\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmprg73oeq9\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmprg73oeq9\model\data\model\assets


13/13 - 3s - loss: 11.8337 - accuracy: 0.7988


C:\Users\yisi9\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpvfuwdqo2\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpvfuwdqo2\model\data\model\assets


In [41]:
import winsound

winsound.Beep(1000, 1000)

In [ ]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=logs